### In this notebook, we exlain how we implement the feature: comparing with reference recipes 

Follow the instruction. https://www.elastic.co/guide/en/elasticsearch/reference/current/getting-started-install.html <br>
pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

In [1]:
from dependency import parent_dir
from common.basics import *
from common.save import save_pickle, load_pickle, save
import spacy

### Preparation

In [3]:
# load the dataset
data = load_pickle('../big_data/data.pickle')
# remember to set-up the localhost first
es = Elasticsearch()

# send data to elasticsearch
for i, recipe in tqdm.tqdm(enumerate(data)):
    recipe['directions'] = recipe.pop('instructions')
    es.index(index='all', doc_type='recipe', body=recipe)

904401it [01:44, 8624.40it/s] 

time: 1min 54s


In [ ]:
'''may need to delete old index
# remember to set-up the localhost first
es = Elasticsearch()
# delete index
es.indices.delete(index='val', ignore=[400, 404])
'''

In [ ]:
'''when we are finally ready

# delete the previous index
es.indices.delete(index='all', ignore=[400, 404])

# write everything to elasticsearch
for i, recipe in tqdm.tqdm(enumerate(data)):
        es.index(index='all', doc_type='recipe', body=recipe)
'''

### Query

In [4]:
def similar(field1, field2, given = 'directions'):
    assert given in ['directions','ingredients']
    field1 = ' '.join(field1) if type(field1) == list else field1
    field2 = ' '.join(field2) if type(field2) == list else field2
    
    query_body = {
      "query": {
        "bool": {
          "must": {
            "multi_match": {   
                "query": field1 + field2,
                'fields':["title", given]
            }
          }
        }
      }
    }
    
    result = es.search(index = 'all', body = query_body)
    best_match = result['hits']['hits'][0]['_source']
    return best_match

time: 26.6 ms


### Example

In [5]:
# prepare example
title = 'blood orange punch'
ingredients = ['orange soda','lime juice','light rum','orange','dash bitters','orange juice']
directions = ['fill ice cube trays with 1 1/2 cups orange juice, and freeze',
              'to make punch, stir remaining orange juice, soda, lime juice, rum, and bitters into a 16-cup punch bowl',
              'before serving, add frozen orange slices',
              'add ice cubes as needed to keep chilled.']

time: 12.9 ms


In [6]:
# example usuage
similar(title, ingredients, given ='ingredients')

{'ingredients': ['dash bitters',
  'lime juice',
  'orange juice',
  'orange',
  'light rum',
  'orange soda'],
 'title': 'blood orange punch',
 'recipe1m_idx': 250985,
 'url': 'http://www.delish.com/recipefinder/blood-orange-punch-recipe',
 'directions': 'fill ice cube trays with 1 1/2 cups orange juice, and freeze. to make punch, stir remaining orange juice, soda, lime juice, rum, and bitters into a 16-cup punch bowl. before serving, add frozen orange slices. add ice cubes as needed to keep chilled.'}

time: 807 ms


In [7]:
# example usuage
similar(title, directions, given ='directions')

{'ingredients': ['dash bitters',
  'lime juice',
  'orange juice',
  'orange',
  'light rum',
  'orange soda'],
 'title': 'blood orange punch',
 'recipe1m_idx': 250985,
 'url': 'http://www.delish.com/recipefinder/blood-orange-punch-recipe',
 'directions': 'fill ice cube trays with 1 1/2 cups orange juice, and freeze. to make punch, stir remaining orange juice, soda, lime juice, rum, and bitters into a 16-cup punch bowl. before serving, add frozen orange slices. add ice cubes as needed to keep chilled.'}

time: 32.3 ms
